# Introduction

This project is an introduction to deep learning for image classification using [Keras](https://keras.io/) and [Tensorflow](https://www.tensorflow.org/) and the [Sequential API](https://keras.io/guides/sequential_model/). In it, I build and optimize a multi-layer perceptron feed-forward neural network to classify images from the [KMINST](http://codh.rois.ac.jp/kmnist/index.html.en) dataset. Then, I implement a convolutional neural network and optimize its layers and parameters.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import itertools

from sklearn import datasets
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier

import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Rescaling
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.optimizers import SGD, RMSprop, Adam

from os import cpu_count

plt.style.use('seaborn-dark')

# Download, Load, Visualize, and Reshape the Data

In [ ]:
X_train = np.load('data/kmnist-train-imgs.npz')['arr_0']
y_train = np.load('data/kmnist-train-labels.npz')['arr_0']
X_test = np.load('data/kmnist-test-imgs.npz')['arr_0']
y_test = np.load('data/kmnist-test-labels.npz')['arr_0']

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

Just like the [MNIST](https://www.tensorflow.org/datasets/catalog/mnist) dataset, we have 60,000 images in the training set and 10,000 images in the test set. Each image is 28x28 pixels.

In [ ]:
print(X_train[0])

Again, just as in the MNIST dataset, the images are grayscale with a value of 0-255 indicating the darkness of the pixel.

In [ ]:
set(y_train)

In [ ]:
sns.histplot(y_train)
plt.title('Training Labels Histogram', fontsize=20)
plt.xlabel('Class', fontsize=15)
plt.ylabel('Count', fontsize=15)

Our labels are 0-9 and we have a very balanced training set.

## Samples of each class
Let's look at several samples of each class.

In [ ]:
samples = 10
label_sample_indexes = dict()
for label in set(y_train):
    label_sample_indexes[label] = [i for i,x in enumerate(y_train) if x==label][0:samples]
label_sample_indexes

In [ ]:
labels = label_sample_indexes.keys()
_, axes = plt.subplots(nrows=len(labels), ncols=samples, figsize=(10, 20))
for ax_row, label in zip(axes, labels):
    for ax, image in zip(ax_row, X_train[label_sample_indexes[label]]):
        ax.set_axis_off()
        ax.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
        ax.set_title(f"Training: {label}")

## Reshaping the Data

Here I will flatten the image itself into a 1D array.

In [ ]:
train_shape = X_train.shape
X_train_flattened = X_train.reshape(train_shape[0], train_shape[1] * train_shape[2])
test_shape = X_test.shape
X_test_flattened = X_test.reshape(test_shape[0], test_shape[1] * test_shape[2])

print(X_train_flattened.shape, X_test_flattened.shape)

In [ ]:
scaler = StandardScaler(with_mean=False, with_std=True)
X_train_scaled = scaler.fit_transform(X_train_flattened)
X_test_scaled = scaler.transform(X_test_flattened)
print("Average:", np.mean(X_train_scaled[:, 350]))
print("Std dev:", np.std(X_train_scaled[:, 350]))

# Classification with Logistic Regression

As in the example, first I'll perform a logistic regression classification to compare, later, with the deep learning models.

In [ ]:
sgd = SGDClassifier(loss="log", n_jobs=cpu_count())
sgd.fit(X_train_scaled, y_train)

y_predicted = sgd.predict(X_test_scaled)

acc = 100. * accuracy_score(y_test, y_predicted)
cm = confusion_matrix(y_test, y_predicted)

print(f"Accuracy: {acc:.2f}")
print("Confusion Matrix:")
print(cm)

The logistic regression performed with an accuracy of about 65%.

# Classification with Multi-Layer Perceptron

For this multiclass classification problem I will choose the [sparse_categorical_crossentropy](https://keras.io/api/losses/probabilistic_losses/#sparsecategoricalcrossentropy-class) for the probabilistic loss function since we have a multiclass problem and our classes are represented as integers (i.e., not one-hot encoded). I'll also begin with the [SGD](https://keras.io/api/optimizers/sgd/) optimizer as I have worked with this the most in class so far.




In [ ]:
def build_model(n_hidden=2, n_neurons=8, dropout=None, loss='sparse_categorical_crossentropy', optimizer=SGD()):
    model = Sequential()
    model.add(Input(shape=(28, 28)))
    model.add(Rescaling(scale=1./255))
    model.add(Flatten())
    for hidden_layer in range(n_hidden):
        model.add(Dense(n_neurons, activation='relu'))
        if dropout:
            model.add(Dropout(dropout))
    model.add(Dense(len(labels), activation='softmax'))
    model.summary()
    model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
    return model


In [ ]:
def fit_and_predict(model, batch_size=256, epochs=10):
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs)
    predicted_probabilities = model.predict(X_test)
    predicted_classes = np.argmax(predicted_probabilities, axis=1)
    acc = 100. * accuracy_score(y_test, predicted_classes)
    cm = confusion_matrix(y_test, predicted_classes)

    return acc, cm

In [ ]:
model = build_model()

In [ ]:
acc, cm = fit_and_predict(model)

In [ ]:
print(f"Accuracy: {acc:.2f}%")
print(f"Confusion Matrix:\n{cm}")

## Optimize the Model

In [ ]:
build_model_params = {
    'n_hidden': [0, 1, 2],
    'n_neurons': [32, 128, 512],
    'dropout': [.1, .2],
    'optimizer': [SGD(), RMSprop(), Adam()]
}
fit_and_predict_params = {
    'epochs': [25, 50]
}

keys, values = zip(*build_model_params.items())
build_model_kwargses = [dict(zip(keys, v)) for v in itertools.product(*values)]
keys, values = zip(*fit_and_predict_params.items())
fit_and_predict_kwargses = [dict(zip(keys, v)) for v in itertools.product(*values)]

results = []
for build_model_kwargs in build_model_kwargses:
    for fit_and_predict_kwargs in fit_and_predict_kwargses:
        model = build_model(**build_model_kwargs)
        acc, cm = fit_and_predict(model, **fit_and_predict_kwargs)
        results.append((build_model_kwargs, fit_and_predict_kwargs, acc, cm))


In [ ]:
best_acc = 0
for result in results:
    acc = result[2]
    if acc > best_acc:
        best_acc = acc
        best_build_model_params = result[0]
        best_fit_and_predict_params = result[1]
        best_cm = result[3]
    print(f"{result[0]} {result[1]} accuracy - {acc:.2f}%")

In [ ]:
print(f"\nBest accuracy: {best_acc:.2f}% - {best_build_model_params} {best_fit_and_predict_params}\n")
print("Best Confusion Matrix:")
print(best_cm)

More hidden layers, more neurons per layer, and the [Adam]() optimizer performed the best. Accuracy is now nearly 93%.

# Implementing a Convolutional Neural Network

For this, I will begin with my best parameters from the multi-layer perceptron model: 2 hidden Dense layers, 512 neurons per dense layer, with a .2 dropout and the [Adam](https://keras.io/api/optimizers/adam/) optimizer.

In [ ]:
def build_cnn_model(
    n_filters=[32, 64], kernel_size=(3, 3), n_hidden=2, pooling=MaxPooling2D(pool_size=(2, 2)),
    n_neurons=512, dropout=.2, loss='sparse_categorical_crossentropy', optimizer=Adam()):
    
    model = Sequential()

    model.add(Input(shape=(28, 28, 1)))
    model.add(Rescaling(scale=1./255))
    
    for filters in n_filters:
        model.add(Conv2D(filters=filters, kernel_size=kernel_size))

    if pooling:
        model.add(pooling)
    
    if pooling is None or type(pooling) is MaxPooling2D:
        model.add(Flatten())

    for hidden_layer in range(n_hidden):
        model.add(Dense(n_neurons, activation='relu'))
        if dropout:
            model.add(Dropout(dropout))
    model.add(Dense(len(labels), activation='softmax'))
    model.summary()
    model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
    return model

In [ ]:
model = build_cnn_model()

In [ ]:
acc, cm = fit_and_predict(model, batch_size=128, epochs=10)

In [ ]:
print(f"Accuracy: {acc:.2f}%")
print(f"Confusion Matrix:\n{cm}")

## Optimize the CNN Model

In [ ]:
build_model_params = {
    'n_filters': [[16, 32], [32, 64], [64, 128]],
    'kernel_size': [(3, 3), (5, 5), (7, 7)],
    'pooling': [None, MaxPooling2D(pool_size=(2, 2)), GlobalMaxPooling2D()],
    'optimizer': [SGD(), RMSprop(), Adam()]
}
fit_and_predict_params = {
    'batch_size': [128, 256],
    'epochs': [10, 25]
}

keys, values = zip(*build_model_params.items())
build_model_kwargses = [dict(zip(keys, v)) for v in itertools.product(*values)]
keys, values = zip(*fit_and_predict_params.items())
fit_and_predict_kwargses = [dict(zip(keys, v)) for v in itertools.product(*values)]

results = []
for build_model_kwargs in build_model_kwargses:
    for fit_and_predict_kwargs in fit_and_predict_kwargses:
        model = build_cnn_model(**build_model_kwargs)
        acc, cm = fit_and_predict(model, **fit_and_predict_kwargs)
        results.append((build_model_kwargs, fit_and_predict_kwargs, acc, cm))

In [ ]:
best_acc = 0
for result in results:
    acc = result[2]
    if acc > best_acc:
        best_acc = acc
        best_build_model_params = result[0]
        best_fit_and_predict_params = result[1]
        best_cm = result[3]
    print(f"{result[0]} {result[1]} accuracy - {acc:.2f}%")

In [ ]:
print(f"\nBest accuracy: {best_acc:.2f}% - {best_build_model_params} {best_fit_and_predict_params}\n")
print("Best Confusion Matrix:")
print(best_cm)